In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

# Import our input dataset
diabetes_df = pd.read_csv('diabetes.csv')
diabetes_df.head()

2022-11-02 17:25:42.439561: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


**Tеперь, когда наш набор данных загружен в Pandas, нам нужно подготовить данные для обучения обеих моделей. С нашей моделью логистической регрессии для данных не требуется предварительная обработка или масштабирование. Однако нашей базовой нейронной сети нужны числовые переменные. Поэтому нам нужно будет отслеживать масштабируемый и немасштабированный тренировочный набор данных, чтобы обе модели имели правильные входные данные в предпочтительных форматах. Чтобы разделить данные, нам нужно добавить и запустить следующий код:**

In [2]:
# Remove diabetes outcome target from features data
y = diabetes_df.Outcome
X = diabetes_df.drop(columns="Outcome")

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

**Next, we need to standardize the numerical variables using Scikit-learn's StandardScaler class. To standardize our data, we need to add and run the following code:**

In [3]:
# Preprocess numerical data for neural network

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

**Теперь мы готовы обучать и оценивать наши модели. Сначала мы начнем с нашей модели логистической регрессии.**

****Для наших целей мы будем использовать наши основные параметры логистической регрессии, которые включают в себя:**
- The solver parameter is set to 'lbfgs', which is an algorithm for learning and optimization. The particular solver isn't very important in this example, but note that a number of optimizers exist.
- The max_iter parameter will be set to 200 iterations, which will give the model sufficient opportunity to converge on effective weights

**Объединив все наши аргументы, мы добавим и запустим следующий код в блокноте:**

In [4]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Logistic regression model accuracy: 0.729


**Затем нам нужно построить, скомпилировать и оценить нашу базовую модель нейронной сети. Опять же, мы будем использовать наши типичные параметры двоичного классификатора:**
- Our single hidden layer will have an input_dim equal to 8, 16 neuron units, and will use the relu activation function.
- Функция потерь должна быть binary_crossentropy, используя оптимизатор adam.
- Our model should provide the additional accuracy scoring metric and train over a maximum of 50 epochs.

**По сравнению с 200 итерациями обучения для нашей модели логистической регрессии, мы будем обучать нашу модель нейронной сети только в течение 50 эпох - это ограничит риск переобучения нашей модели.**

In [5]:
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2022-11-02 17:30:36.277381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
18/18 [==============================] - 1s 2ms/step - loss: 0.8543 - accuracy: 0.5139
Epoch 2/50
18/18 [==============================] - 0s 2ms/step - loss: 0.7865 - accuracy: 0.5608
Epoch 3/50
18/18 [==============================] - 0s 2ms/step - loss: 0.7288 - accuracy: 0.6111
Epoch 4/50
18/18 [==============================] - 0s 2ms/step - loss: 0.6855 - accuracy: 0.6458
Epoch 5/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6466 - accuracy: 0.6510
Epoch 6/50
18/18 [==============================] - 0s 3ms/step - loss: 0.6177 - accuracy: 0.6510
Epoch 7/50
18/18 [==============================] - 0s 3ms/step - loss: 0.5947 - accuracy: 0.6649
Epoch 8/50
18/18 [==============================] - 0s 3ms/step - loss: 0.5752 - accuracy: 0.6944
Epoch 9/50
18/18 [==============================] - 0s 3ms/step - loss: 0.5602 - accuracy: 0.7066
Epoch 10/50
18/18 [==============================] - 0s 3ms/step - loss: 0.5476 - accuracy: 0.7049
Epoch 11/50
18/18 [